In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor, export_graphviz
import graphviz
from IPython.display import Image
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest, chi2
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import RidgeCV, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_absolute_error, classification_report, confusion_matrix
from sklearn.decomposition import PCA
from yellowbrick.regressor import ResidualsPlot


path = '2010-23-data_sa_crime.csv'
original_data = pd.read_csv(path)
data = original_data.copy()

data.head()


columns_to_plot = ["Offence count", "Taken to remand", "Postcode - Incident"]
sns.pairplot(data=data[columns_to_plot])

category_column = "Offence Level 1 Description"
columns_to_plot = ["Offence count", "Taken to remand"]
plt.figure(figsize=(12, 6))
sns.boxplot(x=category_column, y="Offence count", data=data)
plt.title("Box Plot of Offence count by Offence Level 1 Description")
plt.xticks(rotation=90)
plt.show()

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(data["Offence count"], bins=20, edgecolor='k')
plt.xlabel("Offence count")
plt.ylabel("Frequency")
plt.title("Histogram of Offence count")

plt.subplot(1, 2, 2)
plt.hist(data["Taken to remand"], bins=20, edgecolor='k')
plt.xlabel("Taken to remand")
plt.ylabel("Frequency")
plt.title("Histogram of Taken to remand")

plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 6))
plt.scatter(data["Offence count"], data["Taken to remand"], alpha=0.5)
plt.xlabel("Offence count")
plt.ylabel("Taken to remand")
plt.title("Scatter Plot of Offence count vs. Taken to remand")
plt.show()

#***A bar plot can show the average "Offence count" and "Taken to remand" for different values in a categorical column like "Offence Level 2 Description."***
category_column = "Offence Level 2 Description"
columns_to_plot = ["Offence count", "Taken to remand"]
mean_values = data.groupby(category_column)[columns_to_plot].mean()
mean_values.plot(kind="bar", figsize=(12, 6))
plt.xlabel(category_column)
plt.ylabel("Mean Value")
plt.title("Mean Offence count and Taken to remand by " + category_column)
plt.xticks(rotation=90)
plt.show()

#***Histogram of 'Offence count'***

plt.figure(figsize=(8, 4))
sns.histplot(data['Offence count'], bins=20, kde=True)
plt.title('Histogram of Offence Count')
plt.xlabel('Offence Count')
plt.ylabel('Frequency')

#Box Plot of offence

plt.figure(figsize=(8, 4))
sns.boxplot(data['Offence count'])
plt.title('Box Plot of Offence Count')

# Step 2: Check for Missing Values
missing_values = data.isnull().sum()
print("Missing Values:")
print(missing_values)

sns.pairplot(data[['Reported Date', 'Offence count', 'Postcode - Incident']], diag_kind='kde')

data.dropna(inplace=True)
data = pd.get_dummies(data, columns=['Suburb - Incident', 'Offence Level 1 Description', 'Offence Level 2 Description', 'Offence Level 3 Description'])
numeric_columns = ['Postcode - Incident', 'Offence count']



print(data.columns)


offence_level1_columns = [col for col in data.columns if 'Offence Level 1 Description' in col]
offence_level1_counts = data[offence_level1_columns].sum()
plt.figure(figsize=(10, 6))
sns.barplot(x=offence_level1_counts.values, y=offence_level1_counts.index, palette='viridis')
plt.title('Count of Offence Level 1 Description')
plt.xlabel('Count')
plt.ylabel('Offence Level 1 Description')
plt.xticks(rotation=90)
plt.show()


for column in numeric_columns:
    if not pd.api.types.is_numeric_dtype(data[column]):
        # Handle non-numeric values
        data[column] = pd.to_numeric(data[column], errors='coerce')
data.dropna(subset=numeric_columns, inplace=True)


offence_level2_columns = [col for col in data.columns if 'Offence Level 2 Description' in col]
offence_level2_counts = data[offence_level2_columns].sum()
plt.figure(figsize=(10, 6))
sns.barplot(x=offence_level2_counts.values, y=offence_level2_counts.index, palette='magma')
plt.title('Count of Offence Level 2 Description')
plt.xlabel('Count')
plt.ylabel('Offence Level 2 Description')
plt.xticks(rotation=90)
plt.show()


data['Reported Date'] = pd.to_datetime(data['Reported Date'], format='%d/%m/%Y')
data.dropna(inplace=True)


data = original_data.copy()
pos = data[data['Offence Level 1 Description'] == 1]

pos_percentage = len(pos) / len(data)
neg_percentage = 1 - pos_percentage

print('positive instance percentage is', pos_percentage)
print('negative instance percentage is', neg_percentage)


#Violin plot of 'Offence count' vs. 'Offence Level 1 Description'

plt.figure(figsize=(12, 6))
sns.violinplot(data=data, x='Offence Level 1 Description', y='Offence count', palette='Set1')
plt.title('Violin Plot of Offence Count vs. Offence Level 1 Description')
plt.xticks(rotation=90)


data['Reported Date'] = pd.to_datetime(data['Reported Date'], format='%d/%m/%Y')
Y = data['Offence count']

X = data.drop(['Reported Date', 'Offence count'], axis=1)
X = pd.get_dummies(X)
clf = DecisionTreeRegressor(max_depth=3)
scores = cross_val_score(clf, X, Y, cv=5, scoring='r2')
print('Mean R-squared:', scores.mean())

clf.fit(X, Y)
feature_names = list(X.columns)
dot_data = export_graphviz(clf, out_file=None,
                           feature_names=feature_names,
                           filled=True, rounded=True,
                           special_characters=True)

graph = graphviz.Source(dot_data)
graph.render("decision_tree")
Image(graph.render("decision_tree", format='png'))


#Time series plot of 'Reported Date' vs. 'Offence count'


plt.figure(figsize=(12, 6))
sns.lineplot(data=data, x='Reported Date', y='Offence count')
plt.title('Time Series Plot of Offence Count')
plt.xlabel('Reported Date')
plt.ylabel('Offence Count')




plt.figure(figsize=(12, 6))
sns.countplot(data=data, x='Offence Level 1 Description', palette='viridis')
plt.title('Count of Offence Level 1 Description')
plt.xlabel('Offence Level 1 Description')
plt.ylabel('Count')
plt.xticks(rotation=90)


plt.figure(figsize=(12, 6))
sns.countplot(data=data, x='Offence Level 2 Description', palette='magma')
plt.title('Count of Offence Level 2 Description')
plt.xlabel('Offence Level 2 Description')
plt.ylabel('Count')
plt.xticks(rotation=90)


plt.figure(figsize=(12, 6))
sns.barplot(data=data, x='Postcode - Incident', y='Offence count', palette='coolwarm')
plt.title('Mean Offence count by Postcode - Incident')
plt.xlabel('Postcode - Incident')
plt.ylabel('Mean Offence count')
plt.xticks(rotation=90)



#Gausian NB
sample_fraction = 0.1
sampled_data = data.sample(frac=sample_fraction, random_state=42)
Y = sampled_data['Offence count']
X = sampled_data.drop(['Reported Date', 'Offence count'], axis=1)
X = pd.get_dummies(X)

k = 50
selector = SelectKBest(chi2, k=k)
X_selected = selector.fit_transform(X, Y)

# Initialize empty lists for storing results
y = []
x = []

for i in range(1, 16):
    # Create and train a Gaussian Naive Bayes classifier
    clf = GaussianNB()
    scores = cross_val_score(clf, X_selected, Y, cv=10, scoring='accuracy')

    y.append(np.array(scores).mean())
    x.append(i)
plt.plot(x, y)
plt.xlabel('Max Depth')
plt.ylabel('Accuracy')
plt.title('Gaussian Naive Bayes Classifier Accuracy vs. Max Depth')
plt.show()

print('Accuracy Scores:', y)


sample_fraction = 0.1
sampled_data = data.sample(frac=sample_fraction, random_state=42)
Y = sampled_data['Offence count']
X = sampled_data[['Taken to remand']]
y = []

# Create and train a Gaussian Naive Bayes classifier with cross-validation
clf = GaussianNB()
scores = cross_val_score(clf, X, Y, cv=10, scoring='accuracy')
mean_accuracy = np.array(scores).mean()
print(f'Mean Accuracy: {mean_accuracy}')

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)

# Generate a classification report
class_report = classification_report(Y_test, Y_pred)
print("Classification Report:\n", class_report)
conf_matrix = confusion_matrix(Y_test, Y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", linewidths=.5)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

#Pie chart of class distribution

class_distribution = Y.value_counts()
labels = class_distribution.index
sizes = class_distribution.values

plt.figure(figsize=(8, 8))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
plt.axis('equal')
plt.title('Class Distribution')
plt.show()


#Linear SVC
sample_fraction = 0.05
sampled_data = data.sample(frac=sample_fraction, random_state=42)
Y = sampled_data['Offence count']
X = sampled_data.drop(['Reported Date', 'Offence count'], axis=1)
X = pd.get_dummies(X)
y = []
x = []

for i in range(1, 16):
    # Create and train a Linear Support Vector Classifier (LinearSVC)
    clf = LinearSVC(max_iter=1000)
    scores = cross_val_score(clf, X, Y, cv=5, scoring='accuracy')
    y.append(np.array(scores).mean())
    x.append(i)

plt.plot(x, y)
plt.xlabel('Max Depth')
plt.ylabel('Accuracy')
plt.title('Linear Support Vector Classifier Accuracy vs. Max Depth')
plt.show()

print('Accuracy Scores:', y)


plt.figure(figsize=(8, 6))
plt.scatter(x, y, marker='o', color='skyblue')
plt.xlabel('Max Depth')
plt.ylabel('Accuracy')
plt.title('Scatter Plot of Max Depth vs. Accuracy')
plt.grid(True)
plt.show()


plt.figure(figsize=(8, 6))
plt.hist(y, bins=10, color='skyblue', edgecolor='black')
plt.xlabel('Accuracy')
plt.ylabel('Frequency')
plt.title('Histogram of Accuracy')
plt.show()


#RIDGECV

sample_fraction = 0.05
sampled_data = data.sample(frac=sample_fraction, random_state=42)
Y = sampled_data['Offence count']
X = sampled_data.drop(['Reported Date', 'Offence count'], axis=1)
X = pd.get_dummies(X)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Initialize a RidgeCV model with alpha values
alphas = [0.01, 0.1, 1, 10, 100]
clf = RidgeCV(alphas=alphas, store_cv_values=True)

# Fit the RidgeCV model
clf.fit(X_scaled, Y)
optimal_alpha = clf.alpha_
y = []
x = []

for i in range(1, 16):

    clf = Ridge(alpha=optimal_alpha)

    scores = cross_val_score(clf, X_scaled, Y, cv=5, scoring='r2', n_jobs=-1)

    y.append(np.array(scores).mean())
    x.append(i)
plt.plot(x, y)
plt.xlabel('Max Depth')
plt.ylabel('R-squared')
plt.title('Ridge Regression R-squared vs. Max Depth')
plt.show()

print('Optimal Alpha:', optimal_alpha)
print('R-squared Scores:', y)


# Line Plot of R-squared vs. Max Depth
plt.figure(figsize=(10, 6))
plt.plot(x, y, marker='o', linestyle='-', color='b')
plt.xlabel('Max Depth')
plt.ylabel('R-squared')
plt.title('Ridge Regression R-squared vs. Max Depth')
plt.grid(True)
plt.show()


#Ridge Regression

sample_fraction = 0.1
sampled_data = data.sample(frac=sample_fraction, random_state=42)
Y = sampled_data['Offence count']
X = sampled_data.drop(['Reported Date', 'Offence count'], axis=1)
X = pd.get_dummies(X)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Initialize empty lists for storing results
y = []
alphas = [0.1, 1.0, 10.0]

for alpha in alphas:

    clf = Ridge(alpha=alpha)

    scores = cross_val_score(clf, X_scaled, Y, cv=5, scoring='r2', n_jobs=-1)  # Utilize all CPU cores and fewer folds

    y.append(np.array(scores).mean())
plt.plot(alphas, y)
plt.xlabel('Alpha (Regularization Strength)')
plt.ylabel('R-squared')
plt.title('Ridge Regression R-squared vs. Alpha')
plt.show()

print('Best Alpha:', alphas[np.argmax(y)])
print('Best R-squared Score:', max(y))



from sklearn.model_selection import train_test_split
from yellowbrick.regressor import ResidualsPlot

# Split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

# Fit the model on the training data
clf.fit(X_train, Y_train)
visualizer = ResidualsPlot(clf)
visualizer.fit(X_train, Y_train)
visualizer.score(X_test, Y_test)
visualizer.show()



#PCA and KNN

sample_fraction = 0.1
sampled_data = data.sample(frac=sample_fraction, random_state=42)
selected_features = ['Offence count', 'Taken to remand']
X = sampled_data[selected_features]
Y = X['Offence count']
X = X.drop(['Offence count'], axis=1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA for dimensionality reduction
n_components = min(X_scaled.shape[0], X_scaled.shape[1])
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scaled)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X_pca, Y, test_size=0.2, random_state=42)

# Initialize and train a K-NN regressor
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)

# Calculate and print evaluation metrics
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(Y_test, Y_pred)
print(f'Mean Absolute Error: {mae}')
plt.scatter(Y_test, Y_pred)
plt.xlabel('Actual Offence count')
plt.ylabel('Predicted Offence count')
plt.title('Actual vs. Predicted Offence count')
plt.show()


residuals = Y_test - Y_pred
plt.hist(residuals, bins=20)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals')
plt.show()



plt.scatter(Y_pred, residuals)
plt.xlabel('Predicted Offence count')
plt.ylabel('Residuals')
plt.title('Residuals vs. Predicted Offence count')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()



plt.hist(Y, bins=20)
plt.xlabel('Offence count')
plt.ylabel('Frequency')
plt.title('Distribution of Offence Count')
plt.show()



from sklearn.neighbors import KNeighborsRegressor
from yellowbrick.regressor import ResidualsPlot

# Initialize and train a K-NN regressor
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
visualizer = ResidualsPlot(knn)
visualizer.fit(X_train, Y_train)
visualizer.score(X_test, Y_test)
visualizer.show()



# Principal Component Analysis (PCA) Scatter Plot
plt.figure(figsize=(12, 6))
plt.scatter(X_pca[:, 0], Y, c=Y, cmap='viridis')
plt.xlabel('Principal Component 1')
plt.ylabel('Offence count')
plt.title('PCA Scatter Plot')
plt.colorbar(label='Offence count')
plt.show()




from sklearn.decomposition import PCA

# Apply PCA for dimensionality reduction
n_components = min(X_scaled.shape[0], X_scaled.shape[1])
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scaled)

# Explained Variance Ratio
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_explained_variance = explained_variance_ratio.cumsum()
plt.figure(figsize=(12, 6))
plt.bar(range(1, n_components + 1), explained_variance_ratio, alpha=0.5, align='center', label='Explained Variance Ratio')
plt.step(range(1, n_components + 1), cumulative_explained_variance, where='mid', label='Cumulative Explained Variance')
plt.xlabel('Principal Components')
plt.ylabel('Explained Variance Ratio / Cumulative Explained Variance')
plt.title('Scree Plot for PCA')
plt.legend()
plt.show()



#Models comparison

sample_fraction = 0.1
sampled_data = data.sample(frac=sample_fraction, random_state=42)
Y_regression = sampled_data['Offence count']
X_regression = sampled_data.drop(['Reported Date', 'Offence count'], axis=1)
X_regression = pd.get_dummies(X_regression)

# Define your target variable and features for classification
Y_classification = sampled_data['Taken to remand']
X_classification = sampled_data.drop(['Reported Date', 'Taken to remand'], axis=1)
X_classification = pd.get_dummies(X_classification)
regression_results = {}
classification_results = {}

# Standardize features for regression
scaler_regression = StandardScaler()
X_scaled_regression = scaler_regression.fit_transform(X_regression)

# Standardize features for classification
scaler_classification = StandardScaler()
X_scaled_classification = scaler_classification.fit_transform(X_classification)

# Perform PCA for regression
n_components_regression = min(X_scaled_regression.shape[0], X_scaled_regression.shape[1])
pca_regression = PCA(n_components=n_components_regression)
X_pca_regression = pca_regression.fit_transform(X_scaled_regression)

# Split the data into training and testing sets for regression
X_train_regression, X_test_regression, Y_train_regression, Y_test_regression = train_test_split(
    X_pca_regression, Y_regression, test_size=0.2, random_state=42
)

# Perform PCA for classification
n_components_classification = min(X_scaled_classification.shape[0], X_scaled_classification.shape[1])
pca_classification = PCA(n_components=n_components_classification)
X_pca_classification = pca_classification.fit_transform(X_scaled_classification)

# Split the data into training and testing sets for classification
X_train_classification, X_test_classification, Y_train_classification, Y_test_classification = train_test_split(
    X_pca_classification, Y_classification, test_size=0.2, random_state=42
)

# Models for Regression
regression_models = {
    'Ridge Regression': RidgeCV(alphas=[0.01, 0.1, 1, 10, 100], store_cv_values=True),
    'K-NN Regressor': KNeighborsRegressor(n_neighbors=5),
}

# Models for Classification
classification_models = {
    'Random Forest Classifier': RandomForestClassifier(n_estimators=100, random_state=42),
    'Support Vector Classifier': SVC(kernel='linear'),
    'Gaussian Naive Bayes': GaussianNB(),
}

# Train and evaluate regression models
for model_name, model in regression_models.items():
    model.fit(X_train_regression, Y_train_regression)
    Y_pred_regression = model.predict(X_test_regression)
    mae = mean_absolute_error(Y_test_regression, Y_pred_regression)
    regression_results[model_name] = mae

# Train and evaluate classification models
for model_name, model in classification_models.items():
    model.fit(X_train_classification, Y_train_classification)
    Y_pred_classification = model.predict(X_test_classification)
    class_report = classification_report(Y_test_classification, Y_pred_classification)
    conf_matrix = confusion_matrix(Y_test_classification, Y_pred_classification)
    classification_results[model_name] = {'classification_report': class_report, 'confusion_matrix': conf_matrix}

# Visualization 1: Compare MAE of Regression Models
plt.figure(figsize=(12, 6))
plt.bar(regression_results.keys(), regression_results.values())
plt.xlabel('Regression Models')
plt.ylabel('Mean Absolute Error (MAE)')
plt.title('Comparison of Regression Models')
plt.xticks(rotation=45)
plt.show()

# Visualization 2: Classification Report for Classification Models
for model_name, results in classification_results.items():
    print(f"Classification Report for {model_name}:\n{results['classification_report']}")

# Visualization 3: Confusion Matrix Heatmap for Classification Models
plt.figure(figsize=(12, 8))
for i, (model_name, results) in enumerate(classification_results.items(), start=1):
    plt.subplot(2, 2, i)
    sns.heatmap(results['confusion_matrix'], annot=True, fmt="d", cmap="Blues", linewidths=.5)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix for {model_name}')
plt.tight_layout()
plt.show()

# Visualization 4: Residual Plots for Ridge Regression
ridge_regression = regression_models['Ridge Regression']
visualizer = ResidualsPlot(ridge_regression)
visualizer.fit(X_train_regression, Y_train_regression)
visualizer.score(X_test_regression, Y_test_regression)
visualizer.show()

# Visualization 5: PCA Scatter Plot
plt.figure(figsize=(12, 6))
plt.scatter(X_pca_regression[:, 0], Y_regression, c=Y_regression, cmap='viridis')
plt.xlabel('Principal Component 1')
plt.ylabel('Offence count')
plt.title('PCA Scatter Plot')
plt.colorbar(label='Offence count')
plt.show()

# Visualization 6: Principal Component Loadings Plot
plt.figure(figsize=(12, 6))
plt.bar(range(1, n_components_regression + 1), pca_regression.components_[0], label='Principal Component 1')
plt.xlabel('Original Features')
plt.ylabel('Principal Component Loadings')
plt.title('Principal Component Loadings Plot')
plt.legend()
plt.show()









